In [1]:
import numpy as np
import pandas as pd
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math

In [2]:
# Source: fbref.com
# Crawling function that returns DataFrame
def makedf (url, numOfRows = 0):
    
    # Path for the driver 
    PATH = r"C:\Users\Ron\chrome driver\chromedriver.exe"
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    
    # The specific data to analyze
    xp = "/html/body/div[2]/div[6]/div[3]/div[4]/table/tbody"
    table = driver.find_element_by_xpath(xp)
    table_rows = table.find_elements_by_xpath(".//tr")
    
    # Columns for the DataFrame
    name, nation, pos, team, age, born, mp, starts, minutes = [],[],[],[],[],[],[],[],[]
    gls, ast, g_pk, pk, pkatt, crdy, crdr = [],[],[],[],[],[],[]
    
    # Solutiuon for a bug
    i = 25
    numbers = []
    for n in range(30):
        numbers.append(i)
        i += 26
    check = 0
    
    # Number of rows to analayze
    len_table_rows = math.ceil(len(table_rows)-(len(table_rows)/25))
    if numOfRows == 0:
        numOfRows = len_table_rows
    
    # Collecting the data to the DataFrame
    for row in table_rows[:numOfRows]:
        if check not in numbers:
            name.append(row.find_elements_by_class_name("left")[0].text)
            nation.append(row.find_element_by_tag_name("span").text[-3:])
            pos.append(row.find_elements_by_class_name("center")[0].text)
            team.append(row.find_elements_by_class_name("left")[2].text)
            
            if len(row.find_elements_by_class_name("center")[1].text) == 6:
                age.append(int(row.find_elements_by_class_name("center")[1].text[:2]))
            else:
                if row.find_elements_by_class_name("center")[1].text != "":
                    age.append(int(row.find_elements_by_class_name("center")[1].text))
                else:
                    age.append(0)
            
            if row.find_elements_by_class_name("center")[2].text != "":
                born.append(int(row.find_elements_by_class_name("center")[2].text))
            else:
                born.append(0)
            
            if row.find_elements_by_class_name("right")[1].text != "":
                mp.append(int(row.find_elements_by_class_name("right")[1].text))
            else:
                mp.append(0)
                
            if row.find_elements_by_class_name("right")[2].text != "":
                starts.append(int(row.find_elements_by_class_name("right")[2].text))
            else:
                starts.append(0)
                
            if "," in row.find_elements_by_class_name("right")[3].text:
                s = row.find_elements_by_class_name("right")[3].text.replace(",","")
                minutes.append(s)
            else:
                if row.find_elements_by_class_name("right")[3].text != "":
                    minutes.append(int(row.find_elements_by_class_name("right")[3].text))
                else:
                    minutes.append(0)
            
            if row.find_elements_by_class_name("right")[5].text != "":
                gls.append(int(row.find_elements_by_class_name("right")[5].text))
            else:
                gls.append(0)
                
            if row.find_elements_by_class_name("right")[6].text != "":
                ast.append(int(row.find_elements_by_class_name("right")[6].text))
            else:
                ast.append(0)
            
            if row.find_elements_by_class_name("right")[7].text != "":
                g_pk.append(int(row.find_elements_by_class_name("right")[7].text))
            else:
                g_pk.append(0)
            
            if row.find_elements_by_class_name("right")[8].text:
                pk.append(int(row.find_elements_by_class_name("right")[8].text))
            else:
                pk.append(0)
            
            if row.find_elements_by_class_name("right")[9].text != "":
                pkatt.append(int(row.find_elements_by_class_name("right")[9].text))
            else:
                pkatt.append(0)
                
            if row.find_elements_by_class_name("right")[10].text != "":
                crdy.append(int(row.find_elements_by_class_name("right")[10].text))
            else:
                crdy.append(0)
            
            if row.find_elements_by_class_name("right")[11].text != "":
                crdr.append(int(row.find_elements_by_class_name("right")[11].text))
            else:
                crdr.append(0)
                
        check += 1
    
    # Create the DataFrame
    milo = { "Name":name, "Nation":nation, "Position":pos, "Team":team, "Age":age, "Born":born, "Matches Played":mp, 
       "Starts":starts, "Minutes":minutes, "Goals":gls, "Assists":ast, "Non-Penalty Goals":g_pk, 
        "Penalty Kicks Attempted":pk, "Yellow Card":crdy, "Red Cards":crdr }
    df = pd.DataFrame(milo)
    return df
    driver.quit()

# Source 2: Wikipedia
# Data collecting function for 10 different seasons from Wikipedia
def remove_unrelevant(dataframe):
    dataframe.drop(columns=['Pos','Qualification or relegation'],inplace=True)

def tables_pl(year = 2009, two_digit_year = 10):
    list_of_tables = []
    while year<=2018 and two_digit_year<=19:
        url = 'https://en.wikipedia.org/wiki/' + str(year) + '-'+str(two_digit_year)+'_Premier_League'
        old_df = pd.read_html(url)
        df = pd.DataFrame()
        df = old_df[4]
        copy_df = df.copy(deep=False)
        remove_unrelevant(copy_df)
        list_of_tables.append(copy_df)
        year+=1
        two_digit_year+=1
    return list_of_tables

# Function fixing two problematic DataFrame from Wikipedia
def arrange(df):
    df.loc[18,:] = df.loc[20,:]
    df.loc[19,:] = df.loc[21,:]
    df = df.drop([20,21])
    df = df.convert_dtypes("a","e")
    return df

In [3]:
# All the seasons url's from fbref.com
season0910 = "https://fbref.com/en/comps/9/400/stats/2009-2010-Premier-League-Stats"
season1011 = "https://fbref.com/en/comps/9/467/stats/2010-2011-Premier-League-Stats"
season1112 = "https://fbref.com/en/comps/9/534/stats/2011-2012-Premier-League-Stats"
season1213 = "https://fbref.com/en/comps/9/602/stats/2012-2013-Premier-League-Stats"
season1314 = "https://fbref.com/en/comps/9/669/stats/2013-2014-Premier-League-Stats"
season1415 = "https://fbref.com/en/comps/9/733/stats/2014-2015-Premier-League-Stats"
season1516 = "https://fbref.com/en/comps/9/1467/stats/2015-2016-Premier-League-Stats"
season1617 = "https://fbref.com/en/comps/9/1526/stats/2016-2017-Premier-League-Stats"
season1718 = "https://fbref.com/en/comps/9/1631/stats/2017-2018-Premier-League-Stats"
season1819 = "https://fbref.com/en/comps/9/1889/stats/2018-2019-Premier-League-Stats"

# The current ongoing season
season2122 = "https://fbref.com/en/comps/9/stats/Premier-League-Stats"

In [4]:
# Since the crawling for all 10 seasons takes about an hour
# this is an example for one season
example_df = makedf(season0910)
example_df

<ipython-input-2-e5f7ec83c23f>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
<ipython-input-2-e5f7ec83c23f>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = driver.find_element_by_xpath(xp)
C:\Users\Ron\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:426: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\Ron\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\Ron\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_*

,Name,Nation,Position,Team,Age,Born,Matches Played,Starts,Minutes,Goals,Assists,Non-Penalty Goals,Penalty Kicks Attempted,Yellow Card,Red Cards
0,Patrick van Aanholt,NED,DF,Chelsea,18,1990,2,0,35,0,0,0,0,0,0
1,Emmanuel Adebayor,TOG,FW,Manchester City,25,1984,26,25,2207,14,3,14,0,1,0
2,Kieran Agard,ENG,"FW,MF",Everton,19,1989,1,0,1,0,0,0,0,0,0
3,Gabby Agbonlahor,ENG,"FW,MF",Aston Villa,22,1986,36,35,3075,13,3,13,0,3,0
4,Daniel Agger,DEN,DF,Liverpool,24,1984,23,23,1977,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,Stephen Warnock,ENG,DF,Blackburn,27,1981,1,1,90,0,0,0,0,1,0
540,Stephen Warnock,ENG,DF,Aston Villa,27,1981,30,30,2590,0,2,0,0,3,0
541,Ben Watson,ENG,MF,Wigan Athletic,24,1985,5,4,335,1,0,1,0,0,0
542,Danny Webber,ENG,FW,Portsmouth,27,1981,17,4,475,1,1,1,0,1,0


In [5]:
# Crawling from Wikipedia
wiki_tables = tables_pl()

# Fixing some of the data from Wikipedia
wiki_tables[1] = arrange(wiki_tables[1])
wiki_tables[3] = arrange(wiki_tables[3])

wiki_tables[1]

,Team,Pld,W,D,L,GF,GA,GD,Pts
0,Manchester United (C),38,23,11,4,78,37,+41,80
1,Chelsea,38,21,8,9,69,33,+36,71
2,Manchester City,38,21,8,9,60,33,+27,71
3,Arsenal,38,19,11,8,72,43,+29,68
4,Tottenham Hotspur,38,16,14,8,55,46,+9,62
5,Liverpool,38,17,7,14,59,44,+15,58
6,Everton,38,13,15,10,51,45,+6,54
7,Fulham,38,11,16,11,49,43,+6,49
8,Aston Villa,38,12,12,14,48,59,−11,48
9,Sunderland,38,12,11,15,45,56,−11,47


In [6]:
# And DataFrame from Wikipedia
df_wiki_0910 = pd.read_excel("wiki_2009_10.xlsx")
df_wiki_0910

,Unnamed: 0,Team,Pld,W,D,L,GF,GA,GD,Pts
0,0,Chelsea,38,27,5,6,103,32,+71,86
1,1,Manchester Utd,38,27,4,7,86,28,+58,85
2,2,Arsenal,38,23,6,9,83,41,+42,75
3,3,Tottenham,38,21,7,10,67,41,+26,70
4,4,Manchester City,38,18,13,7,73,45,+28,67
5,5,Aston Villa,38,17,13,8,52,39,+13,64
6,6,Liverpool,38,18,9,11,61,35,+26,63
7,7,Everton,38,16,13,9,60,49,+11,61
8,8,Birmingham City,38,13,11,14,38,47,−9,50
9,9,Blackburn,38,13,11,14,41,55,−14,50
